### what is backtesting in quantative finance?

Backtesting is a crucial component in quantitative finance that involves evaluating the performance of a financial model or trading strategy using historical data. The purpose of backtesting is to assess how well a strategy would have performed in the past under various market conditions. This process helps quantitative analysts and traders understand the potential risks and rewards associated with their models before deploying them in real-world markets.

Here's a step-by-step explanation of the backtesting process:

Formulate a Trading Strategy:
Define the rules and logic that your trading strategy will follow. This includes entry and exit signals, position sizing, risk management rules, and any other relevant parameters.

Collect Historical Data:
Gather historical market data that your strategy would have used for decision-making during the specified backtesting period. This data typically includes price and volume information for the relevant financial instruments.

Implement the Strategy:
Write code or use a quantitative finance platform to implement your trading strategy. This involves applying your predefined rules to the historical data to simulate trades and track the hypothetical portfolio's performance.

Simulate Trading Activity:
Execute trades based on the strategy's signals as if you were trading in real time. Keep track of positions, account balances, and any transaction costs or slippage that may occur.

Evaluate Performance:
Assess the strategy's performance by calculating various metrics such as returns, risk-adjusted returns, drawdowns, Sharpe ratio, and other relevant measures. This helps you understand how the strategy would have fared in the past.

Refine and Optimize:
If the backtest results are not satisfactory, refine and optimize the strategy. This may involve adjusting parameters, introducing additional factors, or modifying the logic to improve performance.

Out-of-Sample Testing:
After refining the strategy, conduct out-of-sample testing on a different set of historical data to ensure that the improvements are not simply the result of overfitting to a specific dataset.

Paper Trading or Forward Testing:
Before deploying the strategy in live markets, consider paper trading or forward testing it with real-time data to observe its performance in a simulated real-world environment.

It's important to note that while backtesting is a valuable tool, it has limitations. Past performance is not indicative of future results, and assumptions made during backtesting (such as transaction costs, slippage, and market impact) may not perfectly reflect real-world conditions. Traders and analysts should use backtesting as one part of a broader evaluation process when developing and assessing trading strategies.

### can you show this using python

In [ ]:
import pandas as pd
import yfinance as yf

# Step 2: Collect Historical Data
symbol = "AAPL"
start_date = "2022-01-01"
end_date = "2023-01-01"
historical_data = yf.download(symbol, start=start_date, end=end_date)

# Step 1: Formulate a Trading Strategy
def moving_average_crossover_strategy(data, short_window=50, long_window=200):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0
    
    # Create short simple moving average
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()

    # Create long simple moving average
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()
    
    # Create signals
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)   
    
    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals

# Step 3: Implement the Strategy
signals = moving_average_crossover_strategy(historical_data)

# Step 4: Simulate Trading Activity
# For simplicity, let's assume we invest 100% of our portfolio when the signal is 1 and hold cash otherwise.
initial_capital = 100000.0
positions = pd.DataFrame(index=signals.index).fillna(0.0)
positions[symbol] = 100 * signals['signal']  # 100 shares per buy signal

# Step 5: Evaluate Performance
portfolio = positions.multiply(historical_data['Adj Close'], axis=0)
pos_diff = positions.diff()

# Add cash holdings
portfolio['cash'] = initial_capital - (pos_diff.multiply(historical_data['Adj Close'], axis=0)).cumsum()

# Calculate total portfolio value
portfolio['total'] = portfolio[symbol] + portfolio['cash']

# Plotting
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(portfolio['total'])
plt.title('Portfolio Value over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.show()


### Apply Price Momentum 

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Step 2: Collect Historical Data
symbol = "SPY"
start_date = "2022-01-01"
end_date = "2023-01-01"
historical_data = yf.download(symbol, start=start_date, end=end_date)

# Step 1: Formulate a Trading Strategy (Price Momentum using Rate of Change)
def price_momentum_strategy(data, window=14, momentum_threshold=0):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0
    
    # Calculate the Rate of Change (ROC)
    data['ROC'] = (data['Close'] / data['Close'].shift(window) - 1) * 100
    
    # Create signals based on ROC
    signals['signal'][data['ROC'] > momentum_threshold] = 1.0
    
    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals

# Step 3: Implement the Strategy
momentum_signals = price_momentum_strategy(historical_data)

# Step 4: Simulate Trading Activity
# For simplicity, let's assume we invest 100% of our portfolio when the signal is 1 and hold cash otherwise.
initial_capital = 100000.0
positions = pd.DataFrame(index=momentum_signals.index).fillna(0.0)
positions[symbol] = 100 * momentum_signals['signal']  # 100 shares per buy signal

# Step 5: Evaluate Performance
portfolio = positions.multiply(historical_data['Adj Close'], axis=0)
pos_diff = positions.diff()

# Add cash holdings
portfolio['cash'] = initial_capital - (pos_diff.multiply(historical_data['Adj Close'], axis=0)).cumsum()

# Calculate total portfolio value
portfolio['total'] = portfolio[symbol] + portfolio['cash']

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(portfolio['total'])
plt.title('Portfolio Value over Time - Price Momentum Strategy')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.show()


### Apply Price Momentum for all Asset

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Function to apply price momentum strategy to a list of assets
def apply_momentum_strategy_to_assets(asset_symbols, start_date, end_date, window=14, momentum_threshold=0):
    all_portfolios = {}

    for symbol in asset_symbols:
        # Step 2: Collect Historical Data
        historical_data = yf.download(symbol, start=start_date, end=end_date)

        # Step 3: Formulate a Trading Strategy (Price Momentum using Rate of Change)
        def price_momentum_strategy(data, window=window, momentum_threshold=momentum_threshold):
            signals = pd.DataFrame(index=data.index)
            signals['signal'] = 0.0

            # Calculate the Rate of Change (ROC)
            data['ROC'] = (data['Close'] / data['Close'].shift(window) - 1) * 100

            # Create signals based on ROC
            signals['signal'][data['ROC'] > momentum_threshold] = 1.0

            # Generate trading orders
            signals['positions'] = signals['signal'].diff()

            return signals

        # Step 4: Implement the Strategy
        momentum_signals = price_momentum_strategy(historical_data)

        # Step 5: Simulate Trading Activity
        # For simplicity, let's assume we invest 100% of our portfolio when the signal is 1 and hold cash otherwise.
        initial_capital = 100000.0
        positions = pd.DataFrame(index=momentum_signals.index).fillna(0.0)
        positions[symbol] = 100 * momentum_signals['signal']  # 100 shares per buy signal

        # Evaluate Performance
        portfolio = positions.multiply(historical_data['Adj Close'], axis=0)
        pos_diff = positions.diff()

        # Add cash holdings
        portfolio['cash'] = initial_capital - (pos_diff.multiply(historical_data['Adj Close'], axis=0)).cumsum()

        # Calculate total portfolio value
        portfolio['total'] = portfolio[symbol] + portfolio['cash']

        all_portfolios[symbol] = portfolio

    return all_portfolios

# Example: List of asset symbols
asset_symbols = ["AAPL", "MSFT", "GOOGL"]

# Example: Apply the strategy to the list of assets
#all_portfolios = apply_momentum_strategy_to_assets(asset_symbols, start_date="2022-01-01", end_date="2023-01-01")
all_portfolios = apply_momentum_strategy_to_assets(SPY_tickers, start_date="2022-01-01", end_date="2023-01-01")


# Plotting
plt.figure(figsize=(10, 6))
for symbol, portfolio in all_portfolios.items():
    plt.plot(portfolio['total'], label=symbol)

plt.title('Portfolio Value over Time - Price Momentum Strategy')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.show()


In [ ]:
# import pandas as pd
# import yfinance as yf
# import numpy as np
from sklearn.preprocessing import StandardScaler

# # Step 1: Choose an ETF and identify assets
# chosen_etf_symbol = "SPY"
# chosen_etf = yf.Ticker(chosen_etf_symbol)
# etf_holdings = chosen_etf.get_holdings()

# # Select a subset of assets from the ETF (e.g., first 100)
# assets = etf_holdings.head(100)['Ticker'].tolist()

# # Step 2: Retrieve historical data for the chosen ETF and its assets
# start_date = "2017-01-01"
# end_date = "2022-01-01"

# etf_data = yf.download(chosen_etf_symbol, start=start_date, end=end_date)['Adj Close']
# assets_data = yf.download(assets, start=start_date, end=end_date)['Adj Close']

# Step 3: Calculate the price momentum factors for each asset
def calculate_momentum_factors(data, window=14):
    returns = data.pct_change()
    momentum_factors = returns.rolling(window=window).mean()
    return momentum_factors

asset_momentum_factors = data.groupby(level=0, axis=1).apply(calculate_momentum_factors)

# Step 4: Calculate the monthly z-factor score for each asset
def calculate_z_score(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data.dropna())

asset_z_scores = asset_momentum_factors.groupby(pd.Grouper(freq='M')).apply(calculate_z_score)

# Step 5: Identify long and short baskets using calculated z-scores
def identify_baskets(z_scores, long_count=15, short_count=15):
    long_assets = z_scores.apply(lambda x: x.nlargest(long_count).index.tolist())
    short_assets = z_scores.apply(lambda x: x.nsmallest(short_count).index.tolist())
    return long_assets, short_assets

long_assets, short_assets = identify_baskets(asset_z_scores)

# Step 6: Create a backtest to validate performance
# Note: This is a simple illustration, and a complete backtesting framework would be more complex.

# Assuming equal weights for each asset in the basket
long_returns = data[long_assets.stack().tolist()].pct_change().mean(axis=1)
short_returns = -data[short_assets.stack().tolist()].pct_change().mean(axis=1)

portfolio_returns = long_returns + short_returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Plotting
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns, label='Portfolio')
plt.title('Portfolio Performance - Price Momentum Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()
